In [1]:
import json
import os
import pandas as pd
import numpy as np
from pathlib import Path
import collections
from sklearn.model_selection import train_test_split
from sklearn import metrics

import sys
sys.path.append("../")
from datatools.analyzer import *
from utterance.error_tools import *

from datatools.maneger import DataManager
from datatools.preproc import Preprocessor

In [2]:
from sentence_transformers import SentenceTransformer
from sentence_transformers import models
bert_path = "../../corpus/pretrained/sbert_snli2"
sbert = SentenceTransformer(bert_path)

[28279] 2022-01-21 10:38:03,712 Info sentence_transformers.SentenceTransformer :Load pretrained SentenceTransformer: ../../corpus/pretrained/sbert_snli2
[28279] 2022-01-21 10:38:05,060 Info sentence_transformers.SentenceTransformer :Use pytorch device: cuda


In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn.modules import loss
import torch.optim as optim
import torch.nn.utils.rnn as rnn

In [4]:
def text2vec(text):
    sentences = []
    if isinstance(text, str):
        sentences = [text]
    elif isinstance(text, list):
        sentences = text
    
    return sbert.encode(sentences)

def text2feature(text):
    vector = text2vec(text)
    diff = np.abs( vector[0] - vector[1] )
    return np.concatenate([vector.flatten(), diff])


def vec2feature(vector):
    diff = np.abs( vector[0] - vector[1] )
    return np.concatenate([vector.flatten(), diff])

In [5]:
class SNLIModel(nn.Module):
    def __init__(self, embedding_dim, tagset_size):
        # 親クラスのコンストラクタ。決まり文句
        super(SNLIModel, self).__init__()    
        self.embedding_dim = embedding_dim
        self.hid1= embedding_dim*2
        self.hid2 = embedding_dim//2
        self.fc1 = nn.Linear(self.embedding_dim, self.hid1)
        self.fc2 = nn.Linear(self.hid1, self.hid2)
        # LSTMの出力を受け取って全結合してsoftmaxに食わせるための１層のネットワーク
        # self.hidden2tag = nn.Linear(self.hid2+self.fb_dim, tagset_size)
        self.hidden2tag = nn.Linear(self.hid2, tagset_size)
    
    def forward(self, x):
        # fb = x[:, :self.fb_dim]
        y = F.relu(self.fc1(x))
        # y = F.relu(self.fc1(x[]))
        y = F.relu(self.fc2(y))
        y = self.hidden2tag( y )
        y = F.log_softmax(y, dim=1)
        return y

In [6]:
model_path = "../models/context/"
model_name = "sbert_snli_dnn3.pickle"
modelM = DataManager(model_path)
model = modelM.load_data(model_name)

success load : ../models/context/sbert_snli_dnn3.pickle


In [7]:
symbol = {"unk_token": "[UNK]", "sep_token": "[SEP]", "pad_token": "[PAD]", "cls_token": "[CLS]", "mask_token": "[MASK]"}

In [8]:
# path = "../../corpus/eval_labeled/"
path = "../eval_labeled/"

datalist = ['DCM', 'DIT', 'IRS']
convs = read_conv(path, datalist)

In [9]:
def add_prev_utt(dialogue:list):
    base = ""
    added = []
    for utt in dialogue:
        if base=="":
            added.append(utt)
        else:
            added.append(base+symbol["sep_token"]+utt)
            # added.append(base+utt)
        base = utt
    # 巻き戻す
    added[-1] = dialogue[-1]
    return added

def make_Xy(convs, N=7):
    errors = ["Contradiction", "Self-contradiction"]
    print(errors)
    X = []
    y = []
    for conv in convs:
        dialogue = [""]*N
        for i, ut in enumerate( conv ) :
            # ユーザ発話駆動
            dialogue.append(clean_text( ut.utt) )
            if ut.is_system() and ut.is_exist_error():
                # X.append( add_prev_utt( dialogue[-N:] ))
                X.append(dialogue[-N:])
                if ut.is_error_included(errors) :
                    y.append(1)
                else:
                    y.append(0)
    
    return X, y

In [20]:
N=3
X_str, y = make_Xy(convs, N=N)

['Contradiction', 'Self-contradiction']


In [21]:
X_str_all = sum(X_str, [])

In [22]:
len( X_str_all ) 

4158

In [23]:

X_all = text2vec(X_str_all)

Batches:   0%|          | 0/130 [00:00<?, ?it/s]

In [24]:
X_vec =  X_all.reshape(len(X_str), N, -1) 

In [25]:
X_vec.shape

(1386, 3, 768)

In [26]:
# 推定の時間だ
y_pred = []
for X in tqdm( X_vec):
    hypo = X[-1]
    is_contradict = False
    features = []
    for prev in X[:-1]:
        feature = vec2feature( np.array([prev, hypo]) )
        features.append(feature)
    features = np.array(features)
    with torch.no_grad():
        features = torch.from_numpy(features).cuda()
        y_pred_ = np.array(model(features).cpu()).argmax(axis=1)
    if 2 in y_pred_:
        y_pred.append(1)
    else:
        y_pred.append(0)
   

100%|██████████| 1386/1386 [00:01<00:00, 781.41it/s]


In [27]:
score(y, y_pred)

confusion matrix = 
 [[610 754]
 [ 10  12]]
accuracy =  0.44877344877344877
precision =  0.015665796344647518
recall =  0.5454545454545454
f1 score =  0.03045685279187817


In [28]:
data_path = "../X_y_data/context_content/"
data_name = "contradict.pickle"
dataM = DataManager(data_path)
dataM.save_data(data_name, [y, y_pred])

success save : ../X_y_data/context_content/contradict.pickle


- N=7 eval

        confusion matrix = 
        [[1166  198]
        [  16    6]]
        accuracy =  0.8455988455988456
        precision =  0.029411764705882353
        recall =  0.2727272727272727
        f1 score =  0.05309734513274336

    - 6件だけの検出
    - 結構ミスが多いので，もっとNを減らして効果を見る

- N=6 eval

        confusion matrix = 
        [[1173  191]
        [  16    6]]
        accuracy =  0.8506493506493507
        precision =  0.030456852791878174
        recall =  0.2727272727272727
        f1 score =  0.05479452054794521

    - おそらく最もここが精度が高い，が...

- N=4 eval

        confusion matrix = 
        [[1208  156]
        [  17    5]]
        accuracy =  0.8751803751803752
        precision =  0.031055900621118012
        recall =  0.22727272727272727
        f1 score =  0.054644808743169404

## 発話加算スタイル
- ### 各前提に，直前の発話を加算
- N=6 added

        confusion matrix = 
        [[1256  108]
        [  18    4]]
        accuracy =  0.9090909090909091
        precision =  0.03571428571428571
        recall =  0.18181818181818182
        f1 score =  0.05970149253731343
   
   - 結構誤検出が減る

- N=4 added

        confusion matrix = 
        [[1283   81]
        [  18    4]]
        accuracy =  0.9285714285714286
        precision =  0.047058823529411764
        recall =  0.18181818181818182
        f1 score =  0.07476635514018691

    - 減ったけどなぁ

- N=3 added

    confusion matrix = 
    [[1311   53]
    [  21    1]]
    accuracy =  0.9466089466089466
    precision =  0.018518518518518517
    recall =  0.045454545454545456
    f1 score =  0.02631578947368421

    - N=3, N=4で大きな違いはありそう
    - しかし，未検出の18を拾いたい

In [29]:
# 検出出来ているものはなんだ
i = 0
for i, x in enumerate(X_str):
    if y[i]==0 and y_pred[i]==1:
        print(x)

['こんにちは。気軽に声かけてね。', 'こんにちは。はじめまして。', 'お、そうですか。えっと、四国は行かれたことありますか?']
['兄は無類の麺類好きで、毎日ラーメンを昼食に食べているらしい。', 'ラーメンもいいですね。お兄さんがいらっしゃるんですか?', 'お兄さんに生まれると、頼り甲斐のある性格になるのかな。']
['元気ですかは元気ですね', '今なにしてたの?', 'もちろんですってのが元気ですかにいいですね']
['もちろんですってのが元気ですかにいいですね', 'そうなんだ。どうして?', 'もちろんですってのが元気ですかにいいらしいですよ']
['好きだを見ますよねー', '食べる方がいいよ', '好きだを連発ですよねえ']
['人間関係を深める為の挨拶としてよく用いられるよね。', 'サインはしてもらいますか?', '新しい会社での人間関係が難しくて嫌になっちゃうよ。']
['こんにちは!熱中症に気をつけて。', 'こんにちは。暑いから気をつけないといけませんね', '熱中症に気をつけか?']
['熱中症に気をつけか?', '水をたくさん飲んでいますよ', '熱中症はいいですね']
['熱中症はいいですね', '良くないですよ', 'いつ']
['夏休みは欲しいですね', 'どれ位欲しいですか?', '夏休みは楽しいですね']
['夏休みは楽しいですね', 'ですね。好きです', 'えっ']
['熱中症に気をつけたいのです', '水分とって、涼しくしてください。', '熱中症を引き起こします']
['熱中症を引き起こします', '病院には行きたくないです。', '病院は治療を受けましょう']
['クロールは得意ですね', 'そう、スイミングで習ってるんです。', '朝からスイミングで泳ぎます?']
['スイミングは気持ちが良いですか', 'ええ、ストレス解消になっていいですよ。', 'スイミングを続けるかもしれない']
['何か普段の生活で気をつけていることはありますか。', '健康には特に気を付けています。', '和食、洋食、中華どれが食べたいですか?']
['北朝鮮問題が落ち着きますか?', '時間がかかるでしょうね', 'そういえば、よく旅行に行くと言っていましたが、北朝鮮問題が浮上しますか?']
['北朝鮮が準備を進めるかもしれない

- 検出
        ['', 'こんにちは。熱中症に気をつけて。', 'はい。ありがとう。あなたも気を付けて。', '熱中症に気をつけないんですか?']
        ['こんばんは。それは嬉しいです。', '今日の夕食は済みました?', 'カレーライスを食べました。', '夕食は無しか??']
        ['知りませんでした', '梅雨入りはいいですね', '梅雨明けはいつですか', '梅雨入りは嫌なんですよねえ']
        ['そうですね。特に柴犬が好きです。', 'ちっこい柴犬やと思います', 'そうですか?私は昔柴犬を飼っていました。', 'レトリーバーと柴0匹ってすごいですね。。']
        ['DVDの話だったんだね', 'すごいですね!映画館は久しく行っていないです。', '私も映画館には、とんと御無沙汰です', 'よく映画を見に行くのですか。']
        ['食べてみたいです。', '豆腐も納豆も大好きですね。たくさん食べます。', '私も大好きです。日本人で良かったです。', 'いやいやいや。私好きなんて言ってません。']

- 未検出

        ['行きたいね。なかなか時間が取れないんだけどね。', '服をえらんだりするのはすきですか?', 'センスが良くないから、同じような服ばかり選んでしまうよ。', 'センスはいいですね']
        ['仲間由紀恵', '仲間由紀恵は怖いですね', '何考えてるか分からない感じある', '仲間由紀恵は美しいですね']
        ['どのような活性ですか?', '活性は高いです', '活発なのはいいことです。', '活性が全体的に低いです']
        ['はい。ありがとう。あなたも気を付けて。', '熱中症に気をつけないんですか?', '小まめに水を飲んだりして、気を付けていますよ。', '熱中症に気をつけたいんでしょう?']
        ['小まめに水を飲んだりして、気を付けていますよ。', '熱中症に気をつけたいんでしょう?', 'そうですね。0度倒れていますから、十分気を付けています。', '熱中症に気をつけなければなりませんでしたっけ']
        ['対策はしている積もりですよ。', '対策は大丈夫か??', 'はい。水を飲んだり、塩分もとっています。', '対策は必要ないですか']
        ['なるべく元気でいたいと思っています。', '熱中症が心配されます??', '今日は涼しかったので大丈夫です。', '大丈夫ですか。熱中症が出てますか?']
        ['愛知ですかぁ。いい所ですね。', 'それから名古屋で就職が決まったことですね。', 'どんな仕事をするんですか?', '専業主婦ですよ。']
        ['どこの部位のお肉が好きですか?', 'そうだね。肉のが好きかな。', '私は牛タンが好きなんですよ', 'そうなんですか.牛タンを食べたことはありますか.。']
        ['テニスサークルに入っています。', '入ってます。0人ほどの小さいサークルです。', 'なんのサークルに入ってるのですか?', '今は入っていないです。今は高校時代の友人や、学祭の実行委員をやっていた時の友人とよく遊ぶことが多いです。']
        ['使ってます。', '小さい時くらいしかないです。', 'タブレットも持ってますよ。', '私も持ってないですねぇ・・・。サンタさんに御願いしようかしら。']
        ['そうですね', 'では、音楽はどんなのを聴かれていますか?', '最近はあまり音楽を聴かなくなってきました', 'なるほど。では音楽はどんなジャンルをよく聴きますか?']
        ['あなたも始めてみませんか?', '趣味ですか.因みにあなたの趣味は何ですか?', '音楽鑑賞です', 'なるほど。スポーツが趣味ということですが、何かされていますか?']
        ['', 'こんにちは。最近気になることとか、はまっていることとかありますか?', '私は絵をかくことにはまっています。', 'どんな物を見たのですか?オススメの物があれば教えてください。']
        ['やっぱりそうでしたか', '最終回見ましたか?', '見ました', 'いきなりドラマの話ですか!']
        ['がんばります。今日は暑いですね。', 'いいえ、御弁当を持ってきて院生室で食べました。', 'そうなんですか。私は今から御飯です。', '一緒ですね。']

- 誤検出

        ['', 'こんにちは!熱中症に気をつけて。', 'こんにちは。暑いから気をつけないといけませんね', '熱中症に気をつけか??']
        ['水をたくさん飲んでいますよ', '熱中症はいいですね', '良くないですよ', 'いつ']
        ['', 'こんにちは。海へ行きたいね。', '海はいいですね。泳ぎは得意ですか。', '泳ぎはいいですね']
        ['ありがとう。クロールが得意なんです。', 'クロールは得意ですね', 'そう、スイミングで習ってるんです。', '朝からスイミングで泳ぎます??']
        ['こんにちは。調子はどうですか?', '何か普段の生活で気をつけていることはありますか。', '健康には特に気を付けています。', '和食、洋食、中華どれが食べたいですか?']
        ['センスが良くないから、同じような服ばかり選んでしまうよ。', 'センスはいいですね', '残念だけど、あまり良くないと思うんだ。', '昨日は何を食べましたか?']
        ['所で今日は、何をしてあそんでいますか。', '掛け声をあげるかもしれない', 'わたしはバーベキューをしました。バーベキューは好きですか。', '大声で上げるかもしれない']
        ['わたしはバーベキューをしました。バーベキューは好きですか。', '大声で上げるかもしれない', '大声をあげたら、迷惑になりますよ。雨は好きですか。', '女性が大声を上げるのです']
        ['海に行って何をしたら楽しいですか?', '海は大好きですね', '私も好きです。', 'ありがとう']
        ['洋食ではどんなメニューが好きですか?', '外食はたまにがいいですね', 'そうですね。', 'うむ']
        ['今何をしていますか?', '御覧の通りです。', '一緒にお茶でもどうですか', '独身で0人暮らしですね。']
        ['暇なの?', 'うん', 'じゃあ遊ぼうか', '普段はどんな服を着てますか?']
        ['何か見るのですか?', '池は寒いです', '夏の時期は気持ちが良いかもしれません。', '池に住むのです']
        ['気をつかってくれてありがとうございます。', '熱中症に気をつけてたいですか?', 'なるべく元気でいたいと思っています。', '熱中症が心配されます??']
        ['今からですか?', 'うん', '今日は予定があるので行けません。', '海に行ってないなあ']

---

- 続きの例

        ['夢を追いかけて頑張っている人は素敵ですよね。何か夢はありますか?', 'あせらなくても、大丈夫ですよ。私も、最後まで悩んでいましたから…。', 'ありがとうございます。私もよく悩みますが、ポジティブに考えることは大切ですね。', 'ぎゅうぎゅうですよ。空腹で乗車すると酔うのでガッツリ食べるようにしています。']
        ['ありがとうございます。私もよく悩みますが、ポジティブに考えることは大切ですね。', 'ぎゅうぎゅうですよ。空腹で乗車すると酔うのでガッツリ食べるようにしています。', '私も空腹だと酔います。よく電車に乗られるのですか?', 'それは大変そう!私も高校時代だけバス通学をしていましたが、押されて流されて身動きが取れませんでした。']
        ['私も空腹だと酔います。よく電車に乗られるのですか?', 'それは大変そう!私も高校時代だけバス通学をしていましたが、押されて流されて身動きが取れませんでした。', 'バス通学だったのですね。それだけ混んでいたら学校に着いた時にはヘトヘトですね。', '眠たくても席をゆずります。あはは。']
        ['バス通学だったのですね。それだけ混んでいたら学校に着いた時にはヘトヘトですね。', '眠たくても席をゆずります。あはは。', '素晴らしいですね。尊敬します。ずっと立っているのは辛そうです。', 'なぜですか?']
        ['こんにちは。好きな食べ物はチョコレートです。', '最近チョコにはまって困ってます。', 'どんな種類のチョコレートが好きですか?', '0匹とも雑種ですよ。雉虎と黒猫です']
        ['読書です。', 'どんな本を読みますか?', 'SFが好きです。あなたは?', 'SFですか。映画などもみたりします?']
        ['こんにちは。家ではゲームをしていますよ。', 'ほほう、どのようなゲームをされてるんです', 'FPSが多いですね。あなたは家で何をしているのですか?', '主にパソコンのパーツを見て回っていますね。']
        ['映画を見るには小さいですね', '殆どがシネコンですが。', 'そうなんですか', '机の上が整理されていないと駄目ですね。']
        ['見るだけなら可愛いですよね', '結構楽しいですよ。機会があったらやってみてください。', 'そうなんですか', '机の上が整理されていないと駄目ですね。']
        ['', 'こんにちは。好きな動物は何ですか?', 'こんにちは。私はパンダが大好きです。', 'いやいやいや。私好きなんて言ってません。']